In [1]:
%load_ext line_profiler
%pylab inline
from pycallgraph import PyCallGraph
from pycallgraph.output import GraphvizOutput

from __future__ import print_function
import cProfile

from JSAnimation import IPython_display
from matplotlib import animation
import time
import math

from species import *
from channels import *
from compartment import *
from membrane import *
from params import *

from csdmodel1d import *

import numpy as np
np.seterr(over ='warn',under='ignore')
import scipy as sp
from scipy.integrate import ode

#from dolfin import *  # FEM library
from ProgressBar import *

import logging
logging.getLogger("FFC").setLevel(logging.ERROR)

Populating the interactive namespace from numpy and matplotlib


In [2]:
model = CSDModelInterval(N=100,dx=4e-6) # define the model

# Define the compartments, and the membranes
ecs = Compartment("ecs")
neuron = CellCompartment("neuron",density = 2e5) # 2e5 neurons per meter, 4e10 per sq meter
glia = CellCompartment("glia",density = 2e5) #2e5 glia per meter
#neuron = CellCompartment("neuron",density = 4e10)
#glia = CellCompartment("glia",density = 4e10)

neuron_mem = Membrane("neuronal",inside=neuron,outside=ecs,Cm=Cn,phi_m=-70e-3)
glial_mem = Membrane("glial",inside=glia,outside=ecs,Cm=Cg,phi_m=-85e-3)

# Add the compartments to the model
model.addCompartment(ecs,fraction=0.2) # ECS take 20% of the total volume
model.addCompartment(neuron,fraction=0.4) # Neurons take up 40% of the total volume
model.addCompartment(glia,fraction=0.4) # Neurons take up 40% of the total volume

# Add ion species
ecs.addSpecies(K,Ke0,name='K_e')
ecs.addSpecies(Cl,Cle0,name='Cl_e')
ecs.addSpecies(Na,Nae0,name='Na_e')
ecs.addSpecies(Ca,Cae0,name='Ca_e')

neuron.addSpecies(K,Ki0,0,'K_n')
neuron.addSpecies(Na,Nai0,0,'Na_n')
neuron.addSpecies(Cl,Cli0,0,'Cl_n')
neuron.addSpecies(Ca,Cai0,0,'Ca_n')

glia.addSpecies(K,Ki0,name='K_g')
glia.addSpecies(Na,Nai0,name='Na_g')
glia.addSpecies(Cl,Cli0,name='Cl_g')
glia.addSpecies(Ca,Cai0,0,'Ca_g')

# add channels
neuron_mem.addChannel(NaTChannel(),100.) # 100 per neruon?
neuron_mem.addChannel(KDRChannel(),100.) # number of channels per neuron
neuron_mem.addChannel(KIRChannel(),100.)
neuron_mem.addChannel(NMDAChannel(),300.)
neuron_mem.addChannel(PMCAPump(),500) # PMCA pump
neuron_mem.addChannel(NaCaExchangePump(),500) # sodium-calcium exchanger
neuron_mem.addChannel(NaKATPasePump(),1000.) # 5000 ATPase per neuron
neuron_mem.addChannel(NonSpecificChlorideChannel(-0.07),4000)

glial_mem.addChannel(KIRChannel(),100) # KIR Channel
glial_mem.addChannel(NaKATPasePump(),1000.) # 5000 ATPase per neuron
glial_mem.addChannel(PMCAPump(),500)
glial_mem.addChannel(NonSpecificChlorideChannel(-0.085),4000)

model.addMembrane(neuron_mem)
model.addMembrane(glial_mem)

neuron_mem.addLeakChannels()
neuron.balanceWith(ecs)
glial_mem.addLeakChannels()
glia.balanceWith(ecs)

model.assembleSystem()

Ion: Ca2+, g_leak: 1.96E-05
Ion: Cl-, g_leak: 0.00E+00
Ion: Na+, g_leak: 7.15E-06
Ion: K+, g_leak: 2.76E-05
Ion: Ca2+, g_leak: 1.85E-05
Adding a leak channel does nothing to balance Cl-
Ion: K+, g_leak: 6.15E-05
Ion: Na+, g_leak: 7.25E-06


In [3]:
y = model.getInternalVars()
#%timeit model.ode_rhs(0,y)

In [4]:
cProfile.run('model.ode_rhs(0,y)')

         660 function calls in 0.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
       12    0.000    0.000    0.000    0.000 _methods.py:31(_sum)
       23    0.000    0.000    0.000    0.000 _weakrefset.py:70(__contains__)
       23    0.000    0.000    0.000    0.000 abc.py:128(__instancecheck__)
        5    0.000    0.000    0.000    0.000 channel.py:109(get_h)
        6    0.000    0.000    0.000    0.000 channel.py:125(get_m)
        3    0.000    0.000    0.000    0.000 channel.py:138(conductance)
        3    0.000    0.000    0.000    0.000 channel.py:145(<dictcomp>)
        3    0.000    0.000    0.000    0.000 channel.py:225(mdot)
        2    0.000    0.000    0.000    0.000 channel.py:232(hdot)
        3    0.000    0.000    0.000    0.000 channel.py:309(get_dot_InternalVars)
        7    0.000    0.000    0.000    0.000 channel.py:344(cur

In [5]:
cProfile.run('neuron_mem.phi(y)')

         3 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 membrane.py:231(phi)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [6]:
graphviz = GraphvizOutput()
graphviz.output_file = 'ode_rhs.pdf'
graphviz.output_type = 'pdf'
with PyCallGraph(output=graphviz):
    model.ode_rhs(0,y)

In [7]:
?GraphvizOutput

In [8]:
graphviz2 = GraphvizOutput()
graphviz2.output_file = 'currents.pdf'
graphviz2.output_type = 'pdf'
with PyCallGraph(output=graphviz2):
    neuron_mem.currents(y)

In [9]:
val = ecs.value(K)
val2 = np.zeros(len(val))

In [10]:
%timeit model.ode_rhs(0,y)

100 loops, best of 3: 2.95 ms per loop


In [11]:
len(y)

2500

In [12]:
800/32

25

In [13]:
ecs.tonicity()

array([ 0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,
        0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,
        0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,
        0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,
        0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,
        0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,
        0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,
        0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,
        0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,
        0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,
        0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29,  0.29])

In [14]:
ecs.values.values()

[array([ 0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
         0.001,  0.001,  0.001,  0.001]),
 array([ 0.1455,  0.1455,  0.1455,  0.1455,  0.1455,  0.1455,  0.1455,
         0.1455,  0.145